In [83]:
import urllib
import json
import pandas as pd
import time

In [2]:
# Secret token
token = open("ACCESS_TOKEN.txt", "r").read()

In [91]:
# Format a request URI for the Genius API
search_term = 'Ayla Celik'
_URL_API = "https://api.genius.com/"
_URL_SEARCH = "search?q="
querystring = _URL_API + _URL_SEARCH + urllib.parse.quote(search_term)
request = urllib.request.Request(querystring)
request.add_header("Authorization", "Bearer " + token)
request.add_header("User-Agent", "")

In [61]:
def get_songs():
    try:
        annotations = pd.read_csv('annotations.tsv', sep='\t')
    except:
        return pd.read_csv('song_ids_unique.csv', sep=',')
    songs = pd.read_csv('song_ids_unique.csv', sep=',')
    last_id = str(annotations.song_id.values[-1])
    i = songs[songs.song == last_id].index[0]
    return songs[i+1:]

In [11]:
# Function to recursively search the annotation dict
# and get the text, skipping blockquotes, formatting, images, etc.

def recurse(children):
    st = ''
    for child in children:
        #print(child)
        if type(child) == dict and 'tag' in child:
            if child['tag'] == 'p' or child['tag'] == 'a':
                st += recurse(child['children'])
            elif child['tag'] == 'blockquote':
                st += '[BLOCKQUOTE]'
        elif type(child) == list:
            st += recurse(child)
        elif type(child) == str:
            st += child + ' '
    return st

In [95]:
def append_annotations(songdf, batchsize):
    # Get a chunk of song ids
    song_ids = songdf.head(batchsize).song.astype(int)

    with open('annotations.tsv', 'a') as f:

        for song_id in song_ids:

            # Format the search query using the 'referents' api
            # Given a song id, returns all of its referents (lines that have been annotated)
            # and the corresponding annotations.
            querystring = "https://api.genius.com/referents?song_id=" + str(song_id) + "&per_page=50"
            request = urllib.request.Request(querystring)
            request.add_header("Authorization", "Bearer " + token)
            request.add_header("User-Agent", "")

            # Do internet stuff
            # Send the request to Genius, and parse the response
            try:
                response = urllib.request.urlopen(request, timeout=10)
                string = response.read().decode('utf-8')
                json_obj = json.loads(string)
            except:
                print('***Failed on song id: ' + str(song_id))
                continue

            # The data we want can be found here
            # For this song there are 9 lyric/annotation pairs
            referents = json_obj['response']['referents']

            # Many songs return zero annotations. If we get annotations, parse through them
            if len(referents) > 0:
                for ref in referents:
                    lyric = ref['fragment'].replace('\n', ' ')
                    try:
                        annotation = recurse(ref['annotations'][0]['body']['dom']['children']).replace('\n', ' ')
                    except:
                        print('***Empty annotation on song id: ' + str(song_id))
                        continue
                    
                    print(lyric)
                    f.write(str(song_id) + '\t' + lyric + '\t' + annotation + '\n')
                    
            time.sleep(0.5)

In [97]:
songdf = get_songs()
append_annotations(songdf, 100)

[Verse 2]
Tell me somethin'  You mothafuckas can't tell me nothin'  I'd rather die than to listen to you
I got dark, I got evil, that rot inside my DNA  I got off, I got troublesome heart inside my DNA
[Video Outro]
You ain't shit without a ticket on your plate
My DNA not for imitation  Your DNA an abomination
My expertise checked out in second grade  When I was 9, on cell, motel, we didn't have nowhere to stay  At 29, I've done so well, hit cartwheel in my estate
And pessimists never struck my nerve
And stackin' up the footage, livin' on the go  And sleepin' in a villa  Sippin' from a Grammy, walkin' in the buildin'  Diamond in the ceilin', marble on the floors  Beach inside the window, peekin' out the window
You ain't sick enough to pull it on yourself  You ain't rich enough to hit the lot and skate
I got millions, I got riches buildin' in my DNA
These are the times, level number 9  Look up in the sky, 10 is on the way  Sentence on the way, killings on the way  Motherfucker, I got wi

Ha ha ha, Bad Love, Whaddup?
Cause we all ate cereal
Nipsy got street cred, Jay Rock got street cred  And all I got is bad credit, big ears and big head
And if you feeling froggy to leap, then feel free
Your parody’s not funny
I spaz out 'til I pass out
I came up lower middle class, not poor
I got a buzz like a shot of Palmer Sun
There’s clarity in my vulgarity for a sass-mouth
Ha ha ha, Lady G, Whaddup?
Ali, mix it
The father of the Austin Power fajja
My penmanship is like me and Langston combined to the hip  And my hip to the hop don’t stop as I rock
Get exposed like an R&B ho with a sex tape hidden
Apparently you ain't nothing but industry crash dummies
I’m always for the business, forever you can feel it  Whether you Crip blue or a nigga that hemorrhage
I should: stick a fork in this music business  But I’m too deep in it like pussy on my penis
We tucked the hammers in the hoodies to come and put him away
Hold up homie, you ain't known around here to roll up homie Might get your fa

Soon enough, he became our identity  If you wasn't born and raised here  Or never lived here  You probably would never understand  I'm-a make sure you do
All season has been his goodbye  Him to us, us to him, to the whole world
And nothing would ever replace that or erase that  Forever great in our hearts
So I ask, how do empires last when there's no longer an emperor?  I mean, we got you  But at the same time, we got you
I'm talking about the alleyoop to Shaq
[Sample from untitled 07]
[Sample from untitled 07]
The game four overtime takeover in the Finals
[Announcer]  Kobe, inside the three point arc...  Kobe Bryant has hit a shot at the buzzer!
It's over, it's really over  Kobe really retired, it's crazy!
We were all witnesses
From Compton to Crenshaw
It's just a trip to even be doing this right now  Kobe really retiring
I'm talking about the afro
Growing up watching him paint murals
Using L.A. as his canvas
What can I say?  I mean really, what can we say?  Day by day, season by seas

[Intro: Kendrick Lamar & Chelsea Blythe]
Look, I feel heartless, often off this  Feelin' of fallin', of fallin' apart with  Darkest hours, lost it  Fillin' the void of bein' employed with ballin'
I feel like say somethin', I feel like take somethin'  I feel like skatin' off, I feel like waitin' for 'em  Maybe it's too late for 'em
I can feel it, the phoenix sure to watch us  I can feel it, the dream is more than process  I can put a regime that forms a Loch Ness  I can feel it, the scream that haunts our logic
I feel niggas been out of pocket  I feel niggas tappin' they pockets  I feel like debatin' on who the greatest can stop it
I feel like friends been overrated  I feel like the family been fakin'  I feel like the feelings are changin'  Feel like my thought of compromise is jaded  Feel like you wanna scrutinize how I made it  Feel like I ain't feelin' you all  Feel like removin' myself, no feelings involved  I feel for you, I've been in the field for you  It's real for you, right? S

Abazaba
Fuck that, option number deuce
On point like the helmet of a Viking
Cause she a work-a-art  No help from Julie
It's just you and me  I mean me and her  I mean her and I  It's plain to see, affirmative
Far from an arc, but best undercover
Her body language gets read  Like a ruby
This shit my girlfriend'll like  I ain't tryna entertain these other hoes tonight  I'm Playing shit my girlfriend'll like  Hahaha yea  This shit my girlfriend'll like  I ain't tryna entertain these other hoes tonight  Aba, this shit my girlfriend'll like my nigga
I wrote some shit that my girlfriend'll like  Now I'm sho' to get some cuddy
She do music too, this shit my girlfriend'll write  If she was me talking bout her
You know we get in the studio and do what we do best  You know  Beautiful shit  But this time around I wanna do something different homeboy  You feel me my nig  Hop on them keys real quick my nig
Kodak moment every time we in sync  Like a iPod plugged to a USB
This a great idea  This'll b

...

My flow worth the earth, that's not to be appraised
(Nope) I don't even smoke, I don't even smoke  I don't even smoke, I don't even smoke  I really appreciate that you share your Indo  But a sip of Henny is the farthest I would go  I don't even smoke, I don't even smoke  I don't even smoke, I don't even smoke
Now what you 'bout? Nothing huh
But anyway I think it was some purple  Told me "Don't hit it hard because it'll hurt you"
[Produced by The Drop]
That was the Vegas trip, Hooters sponsored
Bet you think that this some high shit that I wrote  Probably think I'm off the kush or the hydro
The same time I start writing like a fucking monster  I'm Frankenstein every time this mothafucker ponders
That's a jab, you should bob and weave
I didn't listen, I was floating like the Rose Parade  Swear to God a nigga seen some flying fishes
My skill shower over your city for 40 days  40 nights, off the chain, like freedom of slaves  Before you get it twisted, like 40 gays
I stimulate my mind ever

[Verse 3: Kendrick Lamar]
[Chorus: James Fauntleroy]
[Verse 1: Kendrick Lamar]
He's starin' at me in disbelief  My temper is buildin', he's starin' at me, I grab my key  He's starin' at me, I started the car, then I tried to leave  And somethin' told me to keep it in park until I could see  The reason why he was mad at a stranger  Like I was supposed to save him  Like I'm the reason he's homeless and askin' me for a favor  He's starin' at me, his eyes followed me with no laser  He's starin' at me, I notice that his stare is contagious  'Cause now I'm starin' back at him, feelin' some type of disrespect  If I could throw a bat at him, it'd be aimin' at his neck  I never understood someone beggin' for goods  Askin' for handouts, takin' it if they could  And this particular person just had it down pat  Starin' at me for the longest until he finally asked
I should distance myself, I should keep it relentless  My selfishness is what got me here, who the fuck I'm kiddin'?  So I'ma tell you l

Stare at the eyes of Mozart, then tell him I'm the genius
Do my dougie and grab my penis
From Chicago, my daddy and my momma  Came to Compton to accomplish one thing
Ooh wee, that boy remind me of a young Martin Luther  The way he piece up troopers, then round up shooters
Feel the good kid's hunger  The Crips made strong, the Pirus made stronger  Muscle in my heart
Like Malcolm X did, I stand for what I believe in
The meek shall inherit the Earth
Nigga, get out my wallet and put passion in your logic  Passion in your life,
Bright as Thomas Edison, but Gucci man hood
Fucking all my high school teachers  That was '05 bro
105 off-ramp, hey
Got the best product 'round town, Frank Lucas
Little brother only 5, momma put him on my taxes  You know that Compton ghetto shit  That every section 8 tenant is familiar with
501s, Louis letterman
Co-co-co-cock that, aim and shoot that sure shot
I’m still serving, broads on my TIP for gratuities
Burn a swisher up, then I roll again
Don't need a watch t

She got a fat ass with an amazing mind That mean great brain
I swear to God every time I pop off that I pop a vein
See, I inspire conversation with dollar signs You either talking money or just remain the mime
My pockets fat, if not they getting there
I harbor around the orbit on my Darth Vader You'll see the light saber dropping jewels, Marc Jacobs
And you ain't getting it 'til you get in that jew love I'm talking Seinfeld, Ben Stiller, Wall Street, bank tellers
Dominguez Hospital born
I ain't even burn 'em yet  Dave Chapelle these niggas are half baked
That's the mind of state, rap niggas meet your fate  Say hello to the hip hop's Norman Bates
Like CPT, like deep in there, like in the streets
[Part II]
I can just alleviate the rap industry politics
Milk the game up, never lactose intolerant
Zoom, zoom, zoom, zoom, zoom  Zoom, zoom, zoom  Zoom, zoom, zoom, zoom, zoom  Zoom, zoom, zoom  Zoom, zoom, zoom, zoom, zoom  Zoom, zoom, zoom  Zoom, zoom, zoom, shit
She said...
Fuck am I 'posed 

Call me Pharrell, boy, I keep me them Clipse  I don't ride on no skateboard, I roll with some grinders
For example spitting a 16  No thrills, feeling numb like i'm of Thorazine
Patron shots I consume taking the pain away  Each sip bring me closer to my doom
I'm still lost in my own zone in the living room  Like I'm dead to the world playing Donny Hathaway
